In [1]:
import os
import speech_recognition as sr
from gtts import gTTS
from pydub import AudioSegment
from pydub.playback import play
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
load_dotenv()

True

In [12]:
# 텍스트에 색상을 입히기 위한 클래스
class colors:
    RESET = '\033[0m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'

# 텍스트를 음성 파일로 저장하고, pydub으로 재생하는 함수
def speak(text, color=colors.BLUE):
    # 퀴즈 갈매기의 말은 파란색으로 출력
    print(f"{color}🤖 퀴즈 갈매기: {text}{colors.RESET}")
    try:
        tts = gTTS(text=text, lang='ko')
        tts.save('quizbot_response.mp3')

        audio = AudioSegment.from_mp3('quizbot_response.mp3')
        play(audio)
    except Exception as e:
        print(f"음성 출력 중 에러 발생: {e}")
    finally:
        if os.path.exists('quizbot_response.mp3'):
            os.remove('quizbot_response.mp3')

In [13]:
# 사용자의 음성을 텍스트로 변환하는 함수
def listen():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print(f"{colors.YELLOW}🎙️ 듣고 있어요. 말씀해주세요..{colors.RESET}")
        try:
            audio = r.listen(source, timeout=5, phrase_time_limit=10)
            text = r.recognize_google(audio, language='ko-KR')
            print(f"{colors.GREEN}👤 사용자: {text}{colors.RESET}")
            return text
        except sr.UnknownValueError:
            speak("죄송하지만, 잘 알아듣지 못했어요. 다시 말씀해 주시겠어요?", color=colors.RED)
            return ""
        except sr.WaitTimeoutError:
            return "TIMEOUT"

In [15]:
# --- 게임 전체를 감싸는 무한 루프 ---
while True:
    # --- 1. 프로젝트 설정 및 변수 초기화 ---
    client = OpenAI()
    exit_game = False
    player1_name, player2_name = "", ""
    scores, wrong_answers = {}, {}

    # --- 2. 사용자 이름 입력받기 ---
    speak("롯데 자이언츠 2인용 퀴즈쇼에 오신 것을 환영합니다! 참가자 두 분의 이름을 콤마(,)로 구분해서 입력해주세요.")
    try:
        player_names_input = input("[입력] 참가자 이름을 입력하세요 (예: 이대호,손아섭): ")
        names = [name.strip() for name in player_names_input.split(',')]
        if len(names) != 2:
            print("오류: 반드시 2명의 이름을 콤마로 구분하여 입력해야 합니다. 프로그램을 다시 시작해주세요.")
            break
        player1_name, player2_name = names
        scores = {player1_name: 0, player2_name: 0}
        wrong_answers = {player1_name: [], player2_name: []}
        speak(f"반갑습니다, {player1_name}님과 {player2_name}님! 지금부터 퀴즈 대결을 시작하겠습니다!")
    except Exception as e:
        print(f"이름 입력 중 오류가 발생했습니다: {e}")
        break

    # ★★★ 3. AI 퀴즈쇼 호스트 페르소나 정의 및 문제 생성 (프롬프트 강화) ★★★
    system_prompt = """
    당신은 '퀴즈 갈매기'라는 이름의 롯데 자이언츠 퀴즈쇼 호스트입니다.
    '롯데 자이언츠'에 대한 '초급 난이도'의 퀴즈 3개를 문제와 정답 형식으로 JSON 리스트로 만들어주세요.

    #--- 규칙 ---#
    1. ★★★ 매우 중요: 출제하는 모든 퀴즈의 정답은 반드시 검증된 사실(Fact)이어야 하며, 절대 지어내면 안 된다.
    2. 질문은 '선수 등번호', '구단 창단 연도', '우승 연도' 등 누구나 명확하게 확인할 수 있는 주제로만 구성한다.
    3. 주관적이거나 애매한 질문은 절대 하지 않는다.
    4. 마스코트 관련 질문은 절대 하지 않는다.

    #--- JSON 형식 및 좋은 질문 예시 ---#
    [
      {
        "question": "롯데 자이언츠 구단이 창단된 연도는 언제일까요?",
        "answer": "1982년"
      },
      {
        "question": "이대호 선수가 롯데 자이언츠에서 영구결번된 등번호는 몇 번일까요?",
        "answer": "10번"
      }
    ]
    """

    print("AI가 새로운 퀴즈를 출제하는 중입니다...")
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": "규칙에 맞는 퀴즈 3개를 JSON 형식으로 생성해줘."}
            ],
            temperature=0.5,
        )
        import json
        quiz_list = json.loads(response.choices[0].message.content)
        
        # --- 4. 퀴즈 반복문 ---
        for i, quiz in enumerate(quiz_list):
            speak(f"자, {i+1}번째 문제입니다! {quiz['question']}")
            speak("정답을 아시는 분은 먼저 본인의 이름을 외쳐주세요!")
            
            challenger_text = listen()

            if "종료" in challenger_text:
                speak("퀴즈를 종료합니다. 이용해주셔서 감사합니다.")
                exit_game = True
                break
            
            challenger = None
            if player1_name in challenger_text:
                challenger = player1_name
            elif player2_name in challenger_text:
                challenger = player2_name
            
            if challenger:
                speak(f"네, {challenger}님! 답변해주세요.")
                user_answer = listen()

                if user_answer == "TIMEOUT":
                    speak("시간 초과! 아쉽지만 기회가 무효 처리됩니다.")
                elif quiz['answer'].replace(" ", "").lower() in user_answer.replace(" ", "").lower():
                    speak(f"딩동댕! {challenger}님, 정답입니다! 10점 획득!")
                    scores[challenger] += 10
                else:
                    speak(f"아~ 아쉽습니다! 땡! 오답입니다. 정답은 '{quiz['answer']}'이었어요.")
                    wrong_answers[challenger].append({"question": quiz['question'], "correct_answer": quiz['answer'], "my_answer": user_answer})
            else:
                speak(f"아무도 도전하지 않았네요! 이번 문제는 두 분 모두의 오답으로 처리됩니다. 정답은 '{quiz['answer']}'이었어요.")
                wrong_answer_data = {"question": quiz['question'], "correct_answer": quiz['answer'], "my_answer": "도전 안 함"}
                wrong_answers[player1_name].append(wrong_answer_data)
                wrong_answers[player2_name].append(wrong_answer_data)

        if exit_game:
            break

        # --- 5. 최종 결과 발표 ---
        speak("자, 모든 퀴즈가 끝났습니다! 최종 점수를 발표합니다!")
        speak(f"{player1_name}님, {scores[player1_name]}점! {player2_name}님, {scores[player2_name]}점!")
        
        winner = ""
        if scores[player1_name] > scores[player2_name]:
            winner = player1_name
        elif scores[player2_name] > scores[player1_name]:
            winner = player2_name
        
        if winner:
            speak(f"오늘의 퀴즈왕은 바로 {winner}님입니다! 축하합니다!")
        else:
            speak("와, 두 분 다 막상막하! 무승부입니다!")
        
        # --- 6. 사용자별 오답노트 제공 ---
        for player in [player1_name, player2_name]:
            if wrong_answers[player]:
                speak(f"다음은 {player}님의 오답노트입니다. 화면에 띄워드릴 테니 꼭 복습하세요!")
                note = ""
                for idx, item in enumerate(wrong_answers[player]):
                    note += f"{idx+1}번 문제: {item['question']}\n  - 정답: {item['correct_answer']}\n  - 내 답변: {item['my_answer']}\n\n"
                print(f"\n--- 📝 {player}님의 오답노트 ---")
                print(note)
                print("--------------------------------\n")

        # --- 7. 게임 재시작 여부 확인 ---
        speak("퀴즈를 처음부터 다시 풀어보시겠어요? '네' 또는 '아니요'로 대답해주세요.")
        play_again_answer = listen()
        if "네" in play_again_answer or "예" in play_again_answer:
            speak("좋습니다! 그럼 새로운 게임으로 다시 시작하겠습니다!")
            continue
        else:
            speak("알겠습니다! 참여해주셔서 감사합니다. 안녕~")
            break

    except Exception as e:
        print(f"프로그램 실행 중 오류 발생: {e}")
        break

🤖 퀴즈 갈매기: 롯데 자이언츠 2인용 퀴즈쇼에 오신 것을 환영합니다! 참가자 두 분의 이름을 콤마(,)로 구분해서 입력해주세요.
🤖 퀴즈 갈매기: 반갑습니다, 민영님과 송이님! 지금부터 퀴즈 대결을 시작하겠습니다!
AI가 새로운 퀴즈를 출제하는 중입니다...
🤖 퀴즈 갈매기: 자, 1번째 문제입니다! 롯데 자이언츠의 초대 감독은 누구인가요?
🤖 퀴즈 갈매기: 정답을 아시는 분은 먼저 본인의 이름을 외쳐주세요!
🎙️ 듣고 있어요. 말씀해주세요..
🤖 퀴즈 갈매기: 아무도 도전하지 않았네요! 이번 문제는 두 분 모두의 오답으로 처리됩니다. 정답은 '이종범'이었어요.
🤖 퀴즈 갈매기: 자, 2번째 문제입니다! 롯데 자이언츠가 처음으로 한국시리즈에서 우승한 연도는 언제인가요?
🤖 퀴즈 갈매기: 정답을 아시는 분은 먼저 본인의 이름을 외쳐주세요!
🎙️ 듣고 있어요. 말씀해주세요..
🤖 퀴즈 갈매기: 아무도 도전하지 않았네요! 이번 문제는 두 분 모두의 오답으로 처리됩니다. 정답은 '1984년'이었어요.
🤖 퀴즈 갈매기: 자, 3번째 문제입니다! 롯데 자이언츠의 홈구장은 어디인가요?
🤖 퀴즈 갈매기: 정답을 아시는 분은 먼저 본인의 이름을 외쳐주세요!
🎙️ 듣고 있어요. 말씀해주세요..
👤 사용자: 송이
🤖 퀴즈 갈매기: 네, 송이님! 답변해주세요.
🎙️ 듣고 있어요. 말씀해주세요..
👤 사용자: 부산
🤖 퀴즈 갈매기: 아~ 아쉽습니다! 땡! 오답입니다. 정답은 '사직 야구장'이었어요.
🤖 퀴즈 갈매기: 자, 모든 퀴즈가 끝났습니다! 최종 점수를 발표합니다!
🤖 퀴즈 갈매기: 민영님, 0점! 송이님, 0점!
🤖 퀴즈 갈매기: 와, 두 분 다 막상막하! 무승부입니다!
🤖 퀴즈 갈매기: 다음은 민영님의 오답노트입니다. 화면에 띄워드릴 테니 꼭 복습하세요!

--- 📝 민영님의 오답노트 ---
1번 문제: 롯데 자이언츠의 초대 감독은 누구인가요?
  - 정답: 이종범
  - 내 답변: 도전 안 함

2번 문제: 롯데 자이언츠가 처음으로 한국시리즈에서 우승한 연도는 언제인가요